In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))
import pandas as pd
import boto3
from io import StringIO
import io
import string
import random
import json
import pickle
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from scipy import stats
from scipy.stats import norm

# Any results you write to the current directory are saved as output.

### Dataset

- Dataset 2013-1014 time frame
- Train Data: 37 million entires
- Test Data: 2.5 million entries

In [ ]:
# Loading 100k data rows
# Load train data
train = pd.read_csv('../input/train.csv', nrows=100000)

# Load test data
test = pd.read_csv('../input/test.csv', nrows=100000)

# Load destination data
destination = pd.read_csv('../input/destinations.csv', nrows=100000)


In [ ]:
train.head()

In [ ]:
train.columns

In [ ]:
train.info()

#### Features

|  Feature |  Description | 
|----------|:---------|
| date_time  | Timestamp     |
| site_name | ID of Expedia point of sale|
| posa_continent | ID of site’s continent |
| user_location_country |ID of customer’s country |
| user_location_region|ID of customer’s region |
| user_location_city| ID of customer’s city|
| orig_destination_distance| Physical distance between a hotel and a customer|
| user_id| ID of user|
| is_mobile| 1 for mobile device, 0 otherwise|
| is_package| 1 if booking/click was part of package, 0 otherwise|
| channel| ID of a marketing channel|
| srch_ci| Check-in date|
| srch_co| Check-out date|
| srch_adults_cnt| Number of adults|
| srch_children_cnt| Number of children|
| srch_rm_cnt| Number of rooms|
| srch_destination_id| ID of the destination|
| srch_destination_type_id| Type of destination|
| is_booking | 1 if a booking, 0 if a click|
| cnt| Number of similar events in the context of the same user sessiont|
| hotel_continent| Hotel continent|
| hotel_country| Hotel country|
| hotel_market| Hotel market|
| hotel_cluster| ID of hotel cluster|


## Steps

first step was to clean and pre-process the data and perform exploratory analysis to get some interesting insights into the process of choosing a hotel.

- Remove the users who did not booked the hotel
- Identify the searches by each user belonging to a specific type of destination
- orig_destination_distance contains Nan values
- The  check-in  and  check-out  dates  to find the duration of the stay for each of the entries in the training set.



In [ ]:
# Check the percentage of Nan in dataset
total = train.isnull().sum().sort_values(ascending=False)
percent = (train.isnull().sum()/train['hotel_cluster'].count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(20)

## Visualization of Data


In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(15, 10)
sns.heatmap(train.corr(),cmap='coolwarm',ax=ax,annot=True,linewidths=2)

In [ ]:
# Frequency of posa continent
fig, ax = plt.subplots()
fig.set_size_inches(13, 8)
sns.countplot('posa_continent', data=train,order=[0,1,2,3,4],ax=ax)


In [ ]:
# frequency of hotel continent
fig, ax = plt.subplots()
fig.set_size_inches(13, 8)
sns.countplot('hotel_continent', data=train,order=[0,2,3,4,5,6],ax=ax)


In [ ]:
# Frequency of booking through mobile
fig, ax = plt.subplots()
fig.set_size_inches(13, 8)
sns.countplot(x='is_mobile',data=train, order=[0,1],ax=ax)


In [ ]:
# frequency of bookings with package
fig, ax = plt.subplots()
fig.set_size_inches(13, 8)
sns.countplot(x='is_package',data=train, order=[0,1], ax=ax)

## Clean the Data

In [ ]:
train.info()

### Convert it into numerical values which will be relevant to our model.
- date_time
- srch_ci
- srch_co




## Add Extra features
Extract relevant information from date columns
### Additional attributes
- stay_dur: number of duration of stay
- no_of_days_bet_booking: number of days between the booking and 
- Cin_day: Check-in day
- Cin_month: Check-in month
- Cin_year: Check-out year

In [ ]:
# Function to convert date object into relevant attributes
def convert_date_into_days(df):
    df['srch_ci'] = pd.to_datetime(df['srch_ci'])
    df['srch_co'] = pd.to_datetime(df['srch_co'])
    df['date_time'] = pd.to_datetime(df['date_time'])
    
    df['stay_dur'] = (df['srch_co'] - df['srch_ci']).astype('timedelta64[D]')
    df['no_of_days_bet_booking'] = (df['srch_ci'] - df['date_time']).astype('timedelta64[D]')
    
    # For hotel check-in
    # Month, Year, Day
    df['Cin_day'] = df["srch_ci"].apply(lambda x: x.day)
    df['Cin_month'] = df["srch_ci"].apply(lambda x: x.month)
    df['Cin_year'] = df["srch_ci"].apply(lambda x: x.year)


In [ ]:
convert_date_into_days(train)

In [ ]:
train.info()

In [ ]:
# Count the bookings in each month
fig, ax = plt.subplots()
fig.set_size_inches(13, 8)
sns.countplot('Cin_month',data=train[train["is_booking"] == 1],order=list(range(1,13)),ax=ax)

In [ ]:
# Count the bookings as per the day
fig, ax = plt.subplots()
fig.set_size_inches(13, 8)
sns.countplot('Cin_day',data=train[train["is_booking"] == 1],order=list(range(1,32)),ax=ax)

In [ ]:
# Count the bookings as per the stay_duration
fig, ax = plt.subplots()
fig.set_size_inches(13, 8)
sns.countplot('stay_dur',data=train[train["is_booking"] == 1],ax=ax)

In [ ]:
# Check the percentage of Nan in dataset
total = train.isnull().sum().sort_values(ascending=False)
percent = (train.isnull().sum()/train['hotel_cluster'].count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data

### Fill nan with the day which has max occurence


In [ ]:
#train['Cin_day'].value_counts() = 26
#train['Cin_month'].value_counts() = 8
#train['Cin_year'].value_counts() = 2014
#train['stay_dur'].value_counts() = 1
#train['no_of_days_bet_booking'].value_counts() = 0

In [ ]:
train['Cin_day'] = train['Cin_day'].fillna(26.0)
train['Cin_month'] = train['Cin_month'].fillna(8.0)
train['Cin_year'] = train['Cin_year'].fillna(2014.0)
train['stay_dur'] = train['stay_dur'].fillna(1.0)
train['no_of_days_bet_booking'] = train['no_of_days_bet_booking'].fillna(0.0)

In [ ]:
# Fill average values in place for nan, fill with mean
train['orig_destination_distance'].fillna(train['orig_destination_distance'].mean(), inplace=True)

In [ ]:
train.head()

In [ ]:
## Remove datetime object from the dataset
#columns to remove
user_id = train['user_id']
columns = ['date_time', 'srch_ci', 'srch_co','user_id','srch_destination_type_id','srch_destination_id']
train.drop(columns=columns,axis=1,inplace=True)

In [ ]:
train.info()

We have preprocessed our data and it is ready to fit into the model.
All the object values are converted into numerical values. Also, we have more insights of the data.

Reference : http://www.cs.ccsu.edu/~markov/ccsu_courses/datamining-3.html